## Organizing CoinID Data

In [117]:
import pandas as pd
import requests
import json
import ast
from datetime import datetime
import time
format_date = lambda x: datetime.fromtimestamp(int(x)/1000)
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import regex as re
import requests
import xmltodict

In [15]:
df_coinID = pd.read_csv("df_coinID_index_as_columns.csv")

In [16]:
df_coinID.set_index("Unnamed: 0", inplace = True)

In [17]:
df_coinID.rename_axis(None, axis = 0, inplace = True)

In [18]:
df_coinID

,01coin,0-5x-long-algorand-token,0-5x-long-altcoin-index-token,0-5x-long-ascendex-token-token,0-5x-long-bitcoin-cash-token,0-5x-long-bitcoin-token,0-5x-long-cardano-token,0-5x-long-chainlink-token,0-5x-long-cosmos-token,0-5x-long-dogecoin-token,...,polylion,zatcoin,solo-coin,olympus-v1,safeinsure,thekey,mooncake,wolf-game-wool,misty-inu,mintstarter
id,01coin,0-5x-long-algorand-token,0-5x-long-altcoin-index-token,0-5x-long-ascendex-token-token,0-5x-long-bitcoin-cash-token,0-5x-long-bitcoin-token,0-5x-long-cardano-token,0-5x-long-chainlink-token,0-5x-long-cosmos-token,0-5x-long-dogecoin-token,...,polylion,zatcoin,solo-coin,olympus-v1,safeinsure,thekey,mooncake,wolf-game-wool,misty-inu,mintstarter
symbol,zoc,algohalf,althalf,asdhalf,bchhalf,half,adahalf,linkhalf,atomhalf,dogehalf,...,lion,zatcoin,solo,ohm,sins,tky,moon,wool,misty,mint
name,01coin,0.5X Long Algorand Token,0.5X Long Altcoin Index Token,0.5X Long AscendEx Token Token,0.5X Long Bitcoin Cash Token,0.5X Long Bitcoin Token,0.5X Long Cardano Token,0.5X Long Chainlink Token,0.5X Long Cosmos Token,0.5X Long Dogecoin Token,...,PolyLion,ZatCoin,Sologenic,Olympus v1,SafeInsure,THEKEY,MoonCake,Wolf Game Wool,Misty Inu,MintStarter
asset_platform_id,NaN,ethereum,ethereum,ethereum,ethereum,ethereum,ethereum,ethereum,ethereum,ethereum,...,polygon-pos,binance-smart-chain,xrp,ethereum,NaN,neo,binance-smart-chain,ethereum,ethereum,ethereum
platforms,{'': ''},{'ethereum': ''},{'ethereum': ''},{'ethereum': ''},{'ethereum': ''},{'ethereum': ''},{'ethereum': ''},{'ethereum': ''},{'ethereum': ''},{'ethereum': ''},...,{'polygon-pos': '0x1da554d34027ca8de74c5b1cd2f...,{'binance-smart-chain': '0x958e030e5937414b8b5...,{'xrp': ''},{'ethereum': '0x383518188c0c6d7730d91b2c03a03c...,{'': ''},{'neo': '132947096727c84c7f9e076c90f08fec3bc17...,{'binance-smart-chain': '0x2e8c05582176fa93b45...,{'ethereum': '0x8355dbe8b0e275abad27eb843f3eaf...,{'ethereum': '0x814e8f8e35ac8bab913326364ca47b...,{'ethereum': '0x583673b49ee3ae9e59c0b2fa662115...
block_time_in_minutes,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
hashing_algorithm,NeoScrypt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Quark,NaN,NaN,NaN,NaN,NaN
categories,['Masternodes'],[],[],[],[],['Leveraged Token'],[],[],[],[],...,[],['Binance Smart Chain Ecosystem'],['Finance / Banking'],"['Rebase Tokens', 'Decentralized Finance (DeFi...",['Cryptocurrency'],['Big Data'],[],[],[],[]
public_notice,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,"Sologenic is previously named as ""SOLO Coin""",Olympus has migrated to v2: https://www.coinge...,NaN,NaN,NaN,NaN,NaN,The following token has a variable tax functio...
additional_notices,[],[],[],[],[],[],[],[],[],[],...,[],"[""Kindly be aware of <a href='https://www.coin...",[],[],[],[],[],[],[],"[""Kindly be aware of <a href='https://www.coin..."


In [216]:
coin_data = {}
coin_list = []

for coin in df_coinID:
    #converting necessary strings to dict
    coin_market_data = ast.literal_eval(df_coinID[coin]['market_data'])
    coin_community_data = ast.literal_eval(df_coinID[coin]['community_data'])
    coin_description = ast.literal_eval(df_coinID[coin]['description'])
    
    try:
        coin_market_data['ath']['usd']
    except KeyError:
        coin_market_data['ath']['usd'] = np.nan
    
    coin_data = {\
                 # symbols, names and static data
                 'id' : df_coinID[coin]['id'], 
                 'symbol' : df_coinID[coin]['symbol'],
                 'name' : df_coinID[coin]['symbol'],
                 'asset_platform_id' : df_coinID[coin]['asset_platform_id'],
                 'block_time_in_minutes' : df_coinID[coin]['block_time_in_minutes'],
                 'hashing_algorithm' : df_coinID[coin]['hashing_algorithm'],
                 'categories' : df_coinID[coin]['categories'],
                 'description' : coin_description['en'],
                 'country_origin' : df_coinID[coin]['country_origin'],
                 'genesis_date' : df_coinID[coin]['genesis_date'],
                 'sentiment_votes_up_percentage' : df_coinID[coin]['sentiment_votes_up_percentage'],
                 'sentiment_votes_down_percentage' : df_coinID[coin]['sentiment_votes_down_percentage'],
                 'market_cap_rank' : df_coinID[coin]['market_cap_rank'],
                 'coingecko_rank' : df_coinID[coin]['coingecko_rank'],
                 'coingecko_score' : df_coinID[coin]['coingecko_score'],
                 'developer_score' : df_coinID[coin]['developer_score'],
                 'community_score' : df_coinID[coin]['community_score'],
                 'liquidity_score' : df_coinID[coin]['liquidity_score'],
                 'public_interest_score' : df_coinID[coin]['public_interest_score'],
                 # market data
                 'total_value_locked' : coin_market_data['total_value_locked'],
                 'mcap_to_tvl_ratio' : coin_market_data['mcap_to_tvl_ratio'],
                 'fdv_to_tvl_ratio' : coin_market_data['fdv_to_tvl_ratio'],
                 'roi' : coin_market_data['roi'],
                 #market data - ath data
                 'ath_usd' : coin_market_data['ath']['usd'],
                 'ath_btc' : coin_market_data['ath']['btc'],
                 'ath_eth' : coin_market_data['ath']['eth'],
                 'ath_change_percentage_usd' : coin_market_data['ath_change_percentage']['usd'],
                 'ath_change_percentage_btc' : coin_market_data['ath_change_percentage']['btc'],
                 'ath_change_percentage_eth' : coin_market_data['ath_change_percentage']['eth'],
                 'ath_date_usd' : coin_market_data['ath_date']['usd'],
                 'ath_date_btc' : coin_market_data['ath_date']['btc'],
                 'ath_date_eth' : coin_market_data['ath_date']['eth'],
                 #market data - atl data
                 'atl_usd' : coin_market_data['atl']['usd'],
                 'atl_btc' : coin_market_data['atl']['btc'],
                 'atl_eth' : coin_market_data['atl']['eth'],
                 'atl_change_percentage_usd' : coin_market_data['atl_change_percentage']['usd'],
                 'atl_change_percentage_btc' : coin_market_data['atl_change_percentage']['btc'],
                 'atl_change_percentage_eth' : coin_market_data['atl_change_percentage']['eth'],
                 'atl_date_usd' : coin_market_data['atl_date']['usd'],
                 'atl_date_btc' : coin_market_data['atl_date']['btc'],
                 'atl_date_eth' : coin_market_data['atl_date']['eth'],
                 #market data - market cap data
                 'market_cap_usd' : coin_market_data['market_cap']['usd'],
                 'market_cap_btc' : coin_market_data['market_cap']['btc'],
                 'market_cap_eth' : coin_market_data['market_cap']['eth'],
                 'market_cap_rank' : coin_market_data['market_cap_rank'],
                 #community data
                 'facebook_likes' : coin_community_data['facebook_likes'],
                 'twitter_followers' : coin_community_data['twitter_followers'],
                 'reddit_average_posts_48h' : coin_community_data['reddit_average_posts_48h'],
                 'reddit_average_comments_48h' : coin_community_data['reddit_average_comments_48h'],
                 'reddit_subscribers' : coin_community_data['reddit_subscribers'],
                 'reddit_accounts_active_48h' : coin_community_data['reddit_accounts_active_48h'],
                 'telegram_channel_user_count' : coin_community_data['telegram_channel_user_count'],
                 #developers data
                 'developer_data' : df_coinID[coin]['developer_data'],
                 'public_interest_stats' : df_coinID[coin]['public_interest_stats']
                }
    
    #adding coin data to the main list
    coin_list.append(coin_data)

KeyError: 'usd'

In [190]:
coin_market_data = ast.literal_eval(df_coinID[coin]['market_data'])

In [247]:
try:
    print(coin_market_data['ath']['usd'])
except KeyError:
    print("NaN")

NaN


In [249]:
def function_try_except(value):
    try:
        return value
    except KeyError:
        return None

In [250]:
function_try_except(coin_market_data['ath']['usd'])

KeyError: 'usd'

In [245]:
type(ast.literal_eval(df_coinID['coin']['categories']))

list

**Para solucionar**:
- Categories ta como STR, mas preciso converter em list para pegar as categorias mesmo, tem projetos com mais de uma categoria, pensando em pegar a primeiro.
- Transformar STR em dictionary ou json para extrair informacao de market_data.

**Excluidos**:
- public_notice - string de avisos desnecessarios
- additional_notice - string de avisos desnecessarios
- localization - so tem o nome dependendo da localizacao
- description em idiomas que nao sao ingles - nao vai ser util
- link - nao vai ser util
- image - nao vai ser util

In [149]:
coin_df_test = pd.DataFrame(coin_list)

In [151]:
coin_df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13675 entries, 0 to 13674
Data columns (total 23 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   id                               13675 non-null  object
 1   symbol                           13669 non-null  object
 2   name                             13669 non-null  object
 3   asset_platform_id                12139 non-null  object
 4   block_time_in_minutes            13675 non-null  object
 5   hashing_algorithm                1049 non-null   object
 6   categories                       13675 non-null  object
 7   description                      13675 non-null  object
 8   country_origin                   3065 non-null   object
 9   genesis_date                     775 non-null    object
 10  sentiment_votes_up_percentage    5466 non-null   object
 11  sentiment_votes_down_percentage  5466 non-null   object
 12  market_cap_rank                 

<bound method Series.unique of 0        None
1        None
2        None
3        None
4        None
         ... 
13670    None
13671    None
13672    None
13673    None
13674    None
Name: total_value_locked, Length: 13675, dtype: object>